<a href="https://colab.research.google.com/github/d-jenkins/NBA_Champs/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# import all libraries needed
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
import json


In [10]:
url = f'https://www.basketball-reference.com/teams/LAL'

team_stats = pd.read_html(url)[0]

team_stats = team_stats[["Season", "Rel Pace", "Rel ORtg", "Rel DRtg", "Playoffs"]]

for i in range(0,len(team_stats["Playoffs"])):
  # print(team_stats["Playoffs"][i])
  if team_stats["Playoffs"][i] == "Won Finals":
    team_stats["Playoffs"][i] = 1
  else:
    team_stats["Playoffs"][i] = 0

team_stats = team_stats.rename(columns={"Playoffs":"Chip?"})
  
team_stats = team_stats.iloc[0:25, :]

team_stats


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from

,Season,Rel Pace,Rel ORtg,Rel DRtg,Chip?
0,2020-21,-0.5,-2.4,-5.2,0
1,2019-20,0.6,1.4,-4.3,1
2,2018-19,3.2,-2.6,-0.9,0
3,2017-18,3.0,-2.1,-0.6,0
4,2016-17,2.1,-2.8,4.2,0
5,2015-16,-0.2,-4.8,5.2,0
6,2014-15,0.1,-2.2,5.0,0
7,2013-14,4.8,-2.5,3.9,0
8,2012-13,2.4,1.9,0.7,0
9,2011-12,-0.8,1.4,-0.2,0


In [ ]:
# abbreviations of all NBA teams to iterate through
teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 
         'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
         'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
         'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
         'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
         'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

for i in range(0, len(teams)):
  url = f'https://www.basketball-reference.com/teams/{teams[i]}'


ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS


**ETL**

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [6]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NbaChamps").getOrCreate()

In [18]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-06-08 01:37:20--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-06-08 01:37:20 (6.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [20]:
# Convert Pandas df to Pyspark df

team_stats_py = spark.createDataFrame(team_stats)
team_stats_py.printSchema()

root
 |-- Season: string (nullable = true)
 |-- Rel Pace: double (nullable = true)
 |-- Rel ORtg: double (nullable = true)
 |-- Rel DRtg: double (nullable = true)
 |-- Chip?: long (nullable = true)



In [ ]:
select * from 

In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.c6ka6apltccn.us-east-2.rds.amazonaws.com:5432/nba-champs"
config = {"user":"postgres", 
          "password": "MildredChase84!", 
          "driver":"org.postgresql.Driver"}

In [22]:
# Write DataFrame to Nba champs table in RDS

team_stats_py.write.jdbc(url=jdbc_url, table='nba-champs', mode=mode, properties=config)

Py4JJavaError: ignored